# Jupyter Note book MoMo Open API SandBox

In [1]:
from datetime import datetime
Api_User = "" 
Api_Key = ""
GetPaid_Debit_Request_Ref_ID = "" #UUID String Request Reference 
Pay_Transfer_Request_Ref_ID = "" #UUID String Request Reference
Token=""
Token_expiry_time = ""
Token_expired = False
Token_expiry_time = datetime.now()
Environment = "sandbox" #Target Environment  
Collection_Subscription_Primary_Key  = "4c91dae7a6f1474387a23a1f3d448eb7"#Primary Key for Collection Subscription.https://momodeveloper.mtn.com/profile
Base_Url = "https://sandbox.momodeveloper.mtn.com" #SandBox Base URL

<b>Creating API User on the SandBox
This is a one time process.<b> 

In [2]:
#Function to create an API User (Username) from The MoMo OpenAPI SandBox
def Create_API_User_SandBox():
    import requests as rq
    import uuid
    global Api_User
    global Collection_Subscription_Primary_Key
    Api_User = str(uuid.uuid4())
    url = Base_Url+"/v1_0/apiuser"
    headers = {
    "X-Reference-Id": Api_User, #When creating Api user, the X Reference Id in the header will be created as the user. 
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Content-Type": "application/json"
    }
    body = {    
    "providerCallbackHost": "webhook.site" # If your callback is https://webhook.site/mycallback/site then the providerCallbackHost is webhook.site
    }
    try:
        resp = rq.request("post", url, json=body, headers=headers)
        if(str(resp.status_code)=="201"):
            print("HTTP Status Code:"+str(resp.status_code)+"\n Api user Created: "+Api_User)
        elif(str(resp.status_code)=="401"):
            print(str(resp.status_code)+" "+resp.text+" ")
            print("Ensure the subscription key is the primary")
        elif(str(resp.status_code)=="400"):
            print(str(resp.status_code)+" "+resp.text+" ")
            print("Ensure API User(X-Reference-Id) in the Headers is UUID Version 4")
            print("Ensure the Body contains the correct syntax ""\"providerCallbackHost""\""+":"+"Your CallBack URL HOST Eg ""\"webhook.site""\"")
        else:
            print(str(resp.status_code)+" "+resp.text+" ")
    except TypeError:
        print("Body of the Request has to be Json Format")
    except:
        print("Something Is Wrong "+resp.json)
#Create_API_User_SandBox()        

<b>Creating the API Key (Password) For the API USER Created SandBox  {{Api_User}}</b>

In [15]:
#Function to create an API Key for the API User from The MoMo OpenAPI SandBox
def Create_API_Key_SandBox():
    import requests as rq
    import traceback
    global Api_Key
    global Collection_Subscription_Primary_Key
    url = Base_Url+"/v1_0/apiuser/"+Api_User+"/apikey"
    headers = {
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    }    
    try:
        resp = rq.request("post", url, headers=headers)
        if(str(resp.status_code)=="201"):
            Response = resp.json()
            Api_Key = Response.get('apiKey')#Save the API Key in Variable 
            print("HTTP Status Code:"+str(resp.status_code)+"\n Api User:"+str(Api_User) +" Api Key:"+str(Api_Key))
        elif(str(resp.status_code)=="400"):
            print(str(resp.status_code)+" "+resp.text+" Validate the BaseURL \n And Ensure API_User is created, by calling the function Create_API_User_SandBox()")
        elif(str(resp.status_code)=="404"):
            print(str(resp.status_code)+" "+resp.text+" API_USER was not created, Please Run function Create_API_User_SandBox()")
        else:
            print(str(resp.status_code)+" "+resp.text+" ")
    except TypeError:
        print("Body of the Request has to be Json Format or No Body")
    except:
        print("Something Is Wrong ")
        traceback.print_exc()      
#Create_API_Key_SandBox()

#### <b>NOTE: API User and API Key are created once, but advisable to have the API Key changed periodically</b>

In [5]:
Create_API_User_SandBox() #Function create to create API User and store value in Variable {{Api_User}}
Create_API_Key_SandBox() #Function to create API Key(Password) for the Api User Created and stored in variable {{Api_Key}}


HTTP Status Code:201
 Api user Created: 2f096cba-ac57-4cbc-b6a3-ea56821e1bd0
HTTP Status Code:201
 Api User:2f096cba-ac57-4cbc-b6a3-ea56821e1bd0 Api Key:130d62239b6f4b958f8e85f7ef41b827


<b>With the API User and API Key now set up, the next step is to generate a Bearer Token. <br>
This token, which is necessary for authenticating most upcoming API requests, <br>
will expire one hour after its creation.</b>

In [7]:
#Function to generate Token and set Token expiry. 
def Get_Token():# function to return token (renews token if expired)
    import requests as rq
    import traceback
    import json
    from datetime import datetime, timedelta
    global Token 
    global Token_expiry_time
    EndPoint = Base_Url+"/collection/token/"
    Auth = bytes(Api_User + ':' + Api_Key, "utf-8")
    headers = {    
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    }
    try:
        resp = rq.request("post", EndPoint,auth=(Api_User,Api_Key), headers=headers)
        Response = resp.json()
        if(str(resp.status_code) == "200"):
            Token = Response.get('access_token')
            Token_expiry = Response.get('expires_in')
            Token_expiry_time = datetime.now() + timedelta(seconds= int(Token_expiry)) #Track Token Expiry Time 
            print("New Token Generated Expiring at :" +str(Token_expiry_time))           
        elif(str(resp.status_code) == "500" or str(Response.get("error"))=="login_failed"):
            print(Response)
            print("Ensure to Map the API User and API Key as (Username:Password) respectively")
        else:
            print(resp.text)            
    except:
        print("Something Is Wrong ")
        traceback.print_exc()  
#Get_Token()

<b>To prevent the creation of tokens for each request,<br>
 it is advisable to verify the token's validity and only generate a new one when the previous token has expired.</b>

In [11]:
#Function to Validate  Status of Token
#If the Token is Expired a new one will be generated. 
def Token_Status():
    if Token_expiry_time >= datetime.now():
        Token_expired = False    
        print ("Token not Expired: Expiring at "+ str(Token_expiry_time))
        #print(Token)
    else:
        Token_expired = True
        Get_Token()
        #print ("New Token Generated Expiring at "+ str(Token_expiry_time))
        #print(Token)

In [12]:
#Function that initiates a Debit USSD Prompt to the Payer to approve wit PIN
def Request_Debit_Payment(MSISDN,Amount):
  import requests as rq
  from datetime import datetime, timedelta
  import traceback
  import uuid
  global Request_ID_Debit
  Token_Status()
  Request_ID_Debit = str(uuid.uuid4())
  url = Base_Url+"/collection/v1_0/requesttopay"
  headers = {
    "X-Reference-Id": Request_ID_Debit, #Unique for every request, used to validate status of the request. 
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token, #Avoid creating new tokens for every request,  track the Expiry 
    "Content-Type": "application/json",
    "X-Callback-Url":"https://webhook.site/mycallback/site"### You can add X-Callback-Url to receive the callback  ("X-Callback-Url":"https://webhook.com/mysite/status")
  }
  body = {    
    "amount": Amount,
    "currency": "EUR", #use the currency as EUR in the SandBox
    "externalId": str(uuid.uuid1()), #Used for Reconciliation between application and MoMo platform. 
    "payer": {
      "partyIdType": "MSISDN",#EMAIL and ALIAS apply as well 
      "partyId": MSISDN
  },
    "payerMessage": "MoMo Debit API", #Message sent to the Payer
    "payeeNote": "MoMo Debit API" #Message Note to the  Payee
  }
  try:
    resp = rq.request("post", url, json=body, headers=headers)
    if(str(resp.status_code) == "202"):
      print("Debit request to MSISDN "+MSISDN+" Amount "+Amount+" "+ "Response Code "+str(resp.status_code))
      print("Request_Reference_ID :"+Request_ID_Debit )
    elif (str(resp.status_code) == "404"):
      print("Check The Base_URL ")
    elif (str(resp.status_code) == "400"):
      print("Ensure no Special Charters like & in the Message and Notes \nThe X-Reference-Id in the header should be UUID Versio 4")
      print(resp.text)
    elif (str(resp.status_code) == "500" or str(resp.json().get("message")).endswith("INVALID_CALLBACK_URL_HOST") or str(resp.json().get("message")).endswith("Currency not supported.")):
      print(resp.json())
      print("Ensure the  URL Host is the same with the one created when generating API_USer function ")
      print("Verify and validate  Currency for Sand Box is EUR")
    elif (str(resp.status_code) == "500" ):
      print(resp.text)
      print("API is not available")
    else:
      print(resp.status_code)
      print(resp.text)
  except TypeError:
    print("Request Body should be Json Formatted")
  except:
    print("Something Is Wrong ")
    traceback.print_exc()   
#Request_Debit_Payment("56733123453","50000")

<b>From the Test paryer_Id test numbers, we can not validate the status of the Requests </b>

In [13]:
#Check Status 
def Check_Status(X_Reference_Id_Of_The_Debit_Request):
    import requests as rq
    import json
    import traceback
    Token_Status()
    url = Base_Url+"/collection/v1_0/requesttopay/"+X_Reference_Id_Of_The_Debit_Request
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Status_Json = resp.json()
        Status_Json_DD = str(Status_Json).replace('\'', '"')
        print(Status_Json)
        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
    
#Check_Status("1cd09347-8e5a-4f75-8c0f-04a3ecc8c1fa")


#### <b>Testing Status Responses with payer_Id_msisdn</b>
|  payer_id-Msisdn | Response status 
| --- | --- | 
46733123450	|Failed
46733123451	|Rejected
46733123452	|Timeout
56733123453	|Success
46733123454	|Pending

In [14]:

Request_Debit_Payment("56733123452","50000")
Check_Status(Request_ID_Debit)


Token not Expired: Expiring at 2024-10-17 03:16:40.470181
Debit request to MSISDN 56733123452 Amount 50000 Response Code 202
Request_Reference_ID :f34e21f1-4e14-40b7-82a4-57bf9fb866dd
Token not Expired: Expiring at 2024-10-17 03:16:40.470181
{'financialTransactionId': '715428140', 'externalId': 'fe0d9f2c-8c2d-11ef-a65f-7c1e523cf33c', 'amount': '50000', 'currency': 'EUR', 'payer': {'partyIdType': 'MSISDN', 'partyId': '56733123452'}, 'payerMessage': 'MoMo Debit API', 'payeeNote': 'MoMo Debit API', 'status': 'SUCCESSFUL'}
